In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

In [ ]:
df=pd.read_csv("data/Dataset_unsupervised.csv")
df.head(5)

In [ ]:
df.groupby('Camera').describe()

In [ ]:
df_nocam = df[df['Camera']==0]
df_nocam.shape

In [ ]:
df_cam = df[df['Camera']==1]
df_cam.shape

In [ ]:
df_cam_downsampled = df_cam.sample(df_nocam.shape[0])
df_cam_downsampled.shape

In [ ]:
df_balanced = pd.concat([df_cam_downsampled, df_nocam])
df_balanced.shape

In [ ]:
df=df_balanced
df.groupby('Camera').describe()

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df['Command'],df['Camera'], stratify=df['Camera'], test_size=0.1)

In [ ]:
print(len(X_test))
X_train.head(4)


In [ ]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

**Model Type 1**

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

**Model Type 2**

In [ ]:
# BERT layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="the_college_dropout")(outputs['pooled_output'])

# Adding multiple hidden layers
l = tf.keras.layers.Dense(128, activation='relu', name="hidden_layer_1")(l)
l = tf.keras.layers.Dropout(0.2)(l)
l = tf.keras.layers.Dense(64, activation='relu', name="hidden_layer_2")(l)
l = tf.keras.layers.Dropout(0.2)(l)
l = tf.keras.layers.Dense(32, activation='relu', name="hidden_layer_3")(l)

# Output layer
output = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs=[output])


In [ ]:
model.summary()

In [ ]:
len(X_train)

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=20)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
test = ["move the drone 10 m", "move towards the edible object"]
prediction = model.predict(test)
prediction

In [ ]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

In [ ]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

cm = confusion_matrix(y_test, y_predicted)
cm

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sn
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')

In [ ]:
print(classification_report(y_test, y_predicted))